# Bond Pricing project

Bonds are a key financial instrument that are widely used in financial markets as a means to raise capital, they are commonly used by firms and governments. The price of a bond is the present value of the future cash flows, and is therefore sensitive to changes in interest rates. In this project, we will start by looking at bond types and bond ratings, then we will show the code for how to calculate the bond price, yield to maturity and duration. 

### Bond types 

There are several types of different bonds to be aware of:
1. Government bonds - these are issued by governments and are considered to be the safest types of bonds.
2. Corporate bonds - these are issued by firms as a way to raise capital, these offer high yields as a way to compensate for the additional risk.
3. Municipal bonds - these are issued by state and local governments to fund public infrastructure, such as schools, highways and hospitals.
4. High-yield bonds - these are also known as junk bonds, they are issued by companies with a higher risk of default. They offer higher yields to compensate for the additional risk.
5. Convertible bonds - these are bonds which allow investors to convert bonds into a predetermined number of shares of the issuer's stock.
6. Callable bonds - these allow the issuer to redeem the bonds before the maturity, typically at a premium price. 
7. Zero-coupon bonds - these are bonds which do not pay any interest but are sold at a discount to their face value. The return is the difference between the purchase price and the face value. 
8. Floating-rate bonds: These have a variable interest rate that is tied to a benchmark rate such as LIBOR. The interest rate is adjusted periodically to reflect changes in the benchmark rate.

*Note: it is worth taking notice of the fact that not all these bonds are accurately priced by our model, as the bonds we value include interest payments, have a standard coupon rate, no early exercise option and no conversion to shares. For bonds with these factors, we would need to considered alternative models.

### Bond ratings

Bond ratings are an indicator of likelihood of default by the issuer of the bond and are given by bond rating agencies. These agencies look at:
* The strength of the issuer's balance sheet
* Issuer's ability to make debt payments
* The condition of the issuer's operations
* Future economic outlook for the issuer
* The current business conditions
* The strength of the economy

In general, the lower the bond rating, the higher the yield on the bond.

## Importing required libraries

In [92]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Bond cash flow function

In [93]:
def bond_cash_flows(face_value, coupon_rate, maturity, freq):
    periods = maturity * freq 
    coupon_payment = (coupon_rate / freq) * face_value
    coupon_payments = np.repeat(coupon_payment, periods) # Generates array of coupon payments for each period
    coupon_payments[-1] += face_value # Adds face value payment to coupon_payments array
    return coupon_payments

In [94]:
bond1 = bond_cash_flows(100, 0.03, 4, 4)

bond_cash_flows(100, 0.03, 4, 4)

array([  0.75,   0.75,   0.75,   0.75,   0.75,   0.75,   0.75,   0.75,
         0.75,   0.75,   0.75,   0.75,   0.75,   0.75,   0.75, 100.75])

This shows the coupon payments for an 4 year bond that pays four times a year, we can see the final face value payment and coupon payment at the end of our array. 

## Present-value function

In this next section, we generate a present-value function, which calculates the present value of the cash flows from the bond. 

In [95]:
def present_value(cash_flows, yield_rate, freq):
    periods = len(cash_flows) 
    
    # Creates an array which is a repeat of the first argument
    discount_factors = np.repeat(1 + (yield_rate/freq), periods)
    
    # Sets each argument in the array to the negative power of the period number
    discount_factors = np.power(discount_factors, -np.arange(1, periods + 1)) 
    
    # Multiplies the two corresponding values together
    return np.dot(cash_flows, discount_factors)

## Bond yield function

In [96]:
def bond_yield(coupon_rate, face_value, maturity, freq, bond_price, tol=0.0001, max_iter=100):
    
    # Calculates coupon and cash flows
    coupon = (coupon_rate / freq) * face_value
    cash_flows = bond_cash_flows(face_value, coupon_rate, maturity, freq)
    
    # Initial guess for yield
    ytm = coupon /  bond_price
    y_prev = 0.0
    
    # Loop to calculate ytm
    for i in range(max_iter):
        pv = present_value(cash_flows, ytm, freq)
        y_next = ytm - (pv - bond_price) / bond_price * freq / (1 + ytm / freq)
        
        # If condition breaks loop if calculatio is within the tolerance
        if abs(y_next - ytm) < tol:
            ytm = y_next
            break
        else: 
            y_prev = ytm
            ytm = y_next
    
    # Returns yield to maturity
    return ytm

In [97]:
bond_yield(1500, 0.77, 10, 4, 10, tol=1e-6, max_iter=100)

-18646598869.082355

## Creating bond class

In [98]:
class Bond:
    def __init__(self, face_value, coupon_rate, maturity, freq, bond_price):
        self.face_value = face_value
        self.coupon_rate = coupon_rate
        self.maturity = maturity
        self.freq = freq
        self.bond_price = bond_price
        self.cash_flows = bond_cash_flows(face_value, coupon_rate, maturity, freq)
        
    def price(self, yield_rate):
        return present_value(self.cash_flows, yield_rate, self.freq)
    
    def yield_to_maturity(self):
        return bond_yield(self.face_value, self.coupon_rate, self.maturity, self.freq, self.bond_price, tol=1e-6, max_iter=100)
    
    def duration(self, yield_rate):
        periods = self.maturity * self.freq
        discount_factors = np.repeat(1 + (yield_rate / self.freq), periods)
        discount_factors = np.power(discount_factors, -np.arange(1, periods + 1))
        present_values = np.dot(self.cash_flows, discount_factors) 
        weighted_values = present_values * np.arange(1, periods + 1) / self.freq
        return sum(weighted_values) / self.bond_price
        
    def modify_coupon_rate(self, new_coupon_rate):
        self.coupon_rate = new_coupon_rate
        self.cash_flows = bond_cash_flows(self.face_value, self.coupon_rate, self.maturity, self.freq)

    def modify_maturity(self, new_maturity):
        self.maturity = new_maturity
        self.cash_flows = bond_cash_flows(self.face_value, self.coupon_rate, self.maturity, self.freq)

    def modify_yield(self, new_yield):
        self.bond_price = present_value(self.cash_flows, new_yield, self.freq)

In our class, we have methods for calculating the bond price, the bond yield to maturity and the bond duration. The bond price is calculated from the present value of the future cash flows. The bond yield to maturity is calculated by using the Newton-Raphson method, but it can also be calculated from the equation below. Finally, the bond duration is the weighted average times until the bond's cash payments are received, it is a measure for the sensitivity of the bond to changes in the interest rate. 

Bond price equation:


$$
P = \frac{C_1}{(1+y/f)} + \frac{C_2}{(1+y/f)^2} + ... + \frac{C_n + FV}{(1+y/f)^n}
$$


Bond yield to maturity equation:


$$
P = \sum_{t=1}^{n} \frac{C_t}{(1 + y)^t} + \frac{FV}{(1 + y)^n}
$$


Bond duration equation:


$$\text{Duration}=\frac{\sum_{t=1}^T t\times C_t}{P}\times\frac{1}{1+y/f}+\frac{T\times FV}{P\times f\times(1+y/f)^2}$$



## Testing the bond class

In [99]:
# Define parameters
face_value = 1500
coupon_rate = 0.07
maturity = 10
freq = 4
yield_rate = 0.06
bond_price = 1500

In [100]:
bond = Bond(face_value, coupon_rate, maturity, freq, bond_price)

In [101]:
print("Bond price: ", bond.price(0.06))

Bond price:  1612.184419515659


In [102]:
print("Bond yield: ", bond.yield_to_maturity())

Bond yield:  52.80941242246048


In [103]:
print("Bond duration: ", bond.duration(0.06))

Bond duration:  220.33187066714007
